In [2]:
import spacy
import json
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [3]:
f = open('annotations.json')
TRAIN_DATA = json.load(f)

In [4]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 90/90 [00:00<00:00, 919.25it/s]

Skipping entity


In [5]:
##! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

In [6]:
##! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

In [7]:
nlp_ner = spacy.load("model-best")

In [24]:
doc = nlp_ner('''User must register first before proceding into app. After user have an account, user then directed into login page to log into the app using email and password that already registered. After that user will be directed into home page. At home page user will be shown best seller product and recommendation product, user can also browse and search product. Then if user already get product they need, user can proceed to checkout to pay the product. If user have trouble with the payment user can make a report to administrator. Administrator can view and edit user information, cancel payment and setting recommendation product in the home page.
''')
spacy.displacy.render(doc, style="ent", jupyter=True)